## initial mapping workflow for supratidal forests (STF)
- works on geojson from http://geojson.io (add add 'field' with value 1)
- select 'time_range' and if to 'export' results as tif

In [1]:
import time
start = time.time()

In [2]:
%matplotlib inline

import sys
import numpy as np
import xarray as xr
import pandas as pd
import geopandas as gpd
# pd.set_option('display.max_rows', None)

sys.path.insert(1, "/home/jovyan/code/dea-notebooks/Tools")
import datacube
from dea_tools.plotting import display_map, map_shapefile
from datacube.utils.cog import write_cog
from datacube.utils.geometry import Geometry
from dea_tools.spatial import xr_rasterize
dc = datacube.Datacube()

# load virtual product
import importlib
from datacube.virtual import catalog_from_file
from datacube.virtual import DEFAULT_RESOLVER
catalog = catalog_from_file('../virtual_products/virtual_product_cat_wcf.yaml')

### user inputs: geojson AOI, time, export geotiffs?

In [3]:
vector_file = './geojson/ga_summary_grid_c3_coastal.geojson'
attribute_col = 'geometry'

gdf = gpd.read_file(vector_file)

# add time (not a range, just repeat year input here)
time_range = ("2020", "2020")

# export as geotiff?
export = True

In [4]:
mainland_grid = gdf[gdf['type'] == 'mainland']

In [5]:
id_list = []
for index, row in mainland_grid.iterrows():
    id_list.append(row['id'])
print(id_list)

[12, 13, 14, 15, 19, 20, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 38, 39, 40, 41, 42, 43, 44, 45, 48, 49, 50, 51, 52, 53, 54, 56, 57, 58, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 92, 93, 94, 100, 101, 102, 103, 104, 105, 106, 107, 108, 115, 116, 117, 125, 126, 136, 137, 138, 139, 140, 147, 148, 149, 150, 151, 152, 153, 158, 159, 160, 161, 162, 166, 167, 168, 169, 177, 178, 179, 180, 186, 187, 191, 192, 193, 196, 197, 198, 199, 201, 202, 203, 205, 206, 207, 208, 209, 210, 212, 213, 215, 216, 217, 218, 219, 220, 221, 222, 223, 224, 225, 226, 227, 228, 229, 230, 231, 232, 233, 234, 235, 236, 238, 239, 240, 241, 242, 243, 244, 245, 246, 247, 248, 249, 250, 251, 252, 253, 254, 255, 256, 257, 258, 259, 260, 261, 262, 265, 266, 267, 268, 269, 270, 271, 272, 273, 274, 275, 277, 278, 279, 280, 281, 284, 285, 286, 287, 288, 289, 290, 291, 292, 293, 294, 295, 296, 297, 298, 299, 300, 301, 302, 303, 304, 306, 307, 309, 311, 312, 313, 31

In [17]:
# mainland_grid_selection = mainland_grid[268:270]
mainland_grid_selection = mainland_grid.loc[(mainland_grid['id'] >= 101) & (mainland_grid['id'] <= 150)]
mainland_grid_selection

,region_code,ix,iy,utc_offset,id,type,geometry
100,x18y37,18,37,8,101,mainland,"POLYGON ((123.01542 -16.80581, 122.95189 -17.6..."
101,x19y37,19,37,8,102,mainland,"POLYGON ((123.91135 -16.86391, 123.85412 -17.7..."
102,x35y37,35,37,9,103,mainland,"POLYGON ((138.29450 -16.96181, 138.33908 -17.8..."
103,x36y37,36,37,9,104,mainland,"POLYGON ((139.19193 -16.91591, 139.24284 -17.7..."
104,x37y37,37,37,9,105,mainland,"POLYGON ((140.08865 -16.86391, 140.14588 -17.7..."
105,x38y37,38,37,9,106,mainland,"POLYGON ((140.98458 -16.80581, 141.04811 -17.6..."
106,x42y37,42,37,10,107,mainland,"POLYGON ((144.55858 -16.51250, 144.64709 -17.3..."
107,x43y37,43,37,10,108,mainland,"POLYGON ((145.44923 -16.42398, 145.54393 -17.2..."
114,x16y36,16,36,8,115,mainland,"POLYGON ((121.15024 -17.53772, 121.07310 -18.4..."
115,x17y36,17,36,8,116,mainland,"POLYGON ((122.05057 -17.60833, 121.97978 -18.4..."


In [7]:
map_shapefile(mainland_grid_selection, attribute=attribute_col)

/home/jovyan/code/dea-notebooks/Tools/dea_tools/plotting.py:410: FutureWarning: The `map_shapefile` function is deprecated, and will be removed from future versions of `dea-tools`. Please use Geopanda's built-in `.explore` functionality instead.
  warnings.warn(


Label(value='')

Map(center=[-15.348900805509171, 134.19456771615253], controls=(ZoomControl(options=['position', 'zoom_in_text…

In [8]:
# Create the 'query' dictionary object
res = (-30, 30)

query = {
    "time": time_range,
    'resolution':res}

### load in woody cover virtual product

In [9]:
# load in vp woody cover
sys.path.insert(1, '../virtual_products') 
# The dictionary which datacube uses to understand (resolve) the different virtual product functionality
# Need to add any aggregations for the VP you're using

# Get location of transformation
aggregation = "best_pixel_gmad"
agg_loc = importlib.import_module(aggregation)
agg_class = aggregation.split('.')[-1]

DEFAULT_RESOLVER.register('aggregate', agg_class, getattr(agg_loc, agg_class) )

In [10]:
# Need to add any tranformations for the VP you're using

# Get location of transformation
transformation = "woody_cover"
trans_loc = importlib.import_module(transformation)
trans_class = transformation.split('.')[-1]

DEFAULT_RESOLVER.register('transform', trans_class, getattr(trans_loc, trans_class) )

In [11]:
# for index, row in mainland_grid_selection.iterrows():
#     print(str(row['id']))
    
#                       # fname=vector_file.rsplit('/', 1)[-1].split('.')[0] + '_gridID_' + row['id'].astype(str).item() +'_STF_' + time_range[0] + '.tif', # first part gets AOI name


### loop through gdf to derive STF and export

In [ ]:
# Loop through polygons in geodataframe and add geom to queries
for index, row in mainland_grid_selection.iterrows():
    print(f'Feature: {index + 1}/{len(mainland_grid_selection)}')
    
    # Extract the feature's geometry as a datacube geometry object
    geom = Geometry(geom=row.geometry, crs=mainland_grid_selection.crs)
    
    # Update the query to include our geopolygon
    query.update({'geopolygon': geom})

    # Extracting specific keys from dictionary (removing time to load things like item and srtm)
    query_notime = {key: query[key] for key in query.keys()
           & {'resolution', 'geopolygon'}}

    # Load STRM
    srtm_ds = dc.load(product = 'ga_srtm_dem1sv1_0', output_crs="EPSG:3577", **query_notime)
    srtm = srtm_ds.dem_h

    # Load item
    item_ds = dc.load(product = 'item_v2', output_crs="EPSG:3577", **query_notime)
    item = item_ds.relative

    # Load in mangrove cover
    DEAmangrove = dc.load(product = 'ga_ls_mangrove_cover_cyear_3', output_crs="EPSG:3577", **query)

    # if no mangroves within AOI, create dummy xr.dataarray
    if DEAmangrove.data_vars == {}:
        mangrove = xr.DataArray(np.zeros_like(srtm), coords=srtm.coords, dims=srtm.dims, attrs=srtm.attrs)
    else:
        # get output of mangrove == 1, not mangrove == 0
        mangrove = (DEAmangrove.canopy_cover_class != 255)


    # load woody_cover vp
    product = catalog['woody_cover']
    woody_cover_vp = product.load(dc, **query)

    # woody cover with threshold for saltmarsh
    woody_cover = xr.where(woody_cover_vp.woody_cover > 0.5, 1, 0).astype('int8')
    woody_cover.attrs['crs'] = 'EPSG:3577'

    ## wrangle binary maps and create STF mask

    # not mangrove == True
    not_mangrove = (1 - mangrove)
    not_mangrove = not_mangrove == 1

    # greater than 1m AHD and less than 10m AHD == True
    AHD_min = 1
    AHD_max = 10

    # SRTM
    lessthan10m_AHD = srtm <= AHD_max
    greaterthan1m_AHD = srtm >= AHD_min
    AHD_threshold_srtm = lessthan10m_AHD & greaterthan1m_AHD

    # item (remove all intertidal areas)
    not_intertidal = item == 9

    # remove time dim
    not_mangrove = not_mangrove.squeeze('time')
    AHD_threshold_srtm = AHD_threshold_srtm.squeeze('time')
    not_intertidal = not_intertidal.squeeze('time')

    # combine masks
    STF = xr.where((woody_cover == False) + (not_mangrove == False) + (AHD_threshold_srtm == False) + (not_intertidal == False), 0, 1).astype('int8')

    # Generate a polygon mask to keep only data within the polygon
    mask = xr_rasterize(mainland_grid_selection, srtm_ds)

    # Mask dataset to set pixels outside the polygon to `NaN`
    STF = STF.where(mask)


    # STF (1 = STF, 0 = not STF)
    if export == False:
        pass
    else:
        write_cog(geo_im=STF.astype('int8'),
                  fname=vector_file.rsplit('/', 1)[-1].split('.')[0] + '_gridID_' + str(row['id']) +'_STF_' + time_range[0] + '.tif', # first part gets AOI name
                  overwrite=True,
                  nodata=0.0)

Feature: 51/37


/env/lib/python3.8/site-packages/sklearn/base.py:299: UserWarning: Trying to unpickle estimator DecisionTreeRegressor from version 1.0.2 when using version 1.2.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/env/lib/python3.8/site-packages/sklearn/base.py:299: UserWarning: Trying to unpickle estimator RandomForestRegressor from version 1.0.2 when using version 1.2.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/env/lib/python3.8/site-packages/sklearn/base.py:420: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(


Feature: 52/37


/env/lib/python3.8/site-packages/sklearn/base.py:299: UserWarning: Trying to unpickle estimator DecisionTreeRegressor from version 1.0.2 when using version 1.2.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/env/lib/python3.8/site-packages/sklearn/base.py:299: UserWarning: Trying to unpickle estimator RandomForestRegressor from version 1.0.2 when using version 1.2.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/env/lib/python3.8/site-packages/sklearn/base.py:420: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(


Feature: 53/37


...

/env/lib/python3.8/site-packages/sklearn/base.py:299: UserWarning: Trying to unpickle estimator DecisionTreeRegressor from version 1.0.2 when using version 1.2.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/env/lib/python3.8/site-packages/sklearn/base.py:299: UserWarning: Trying to unpickle estimator RandomForestRegressor from version 1.0.2 when using version 1.2.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/env/lib/python3.8/site-packages/sklearn/base.py:420: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(


Feature: 94/37


/env/lib/python3.8/site-packages/sklearn/base.py:299: UserWarning: Trying to unpickle estimator DecisionTreeRegressor from version 1.0.2 when using version 1.2.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/env/lib/python3.8/site-packages/sklearn/base.py:299: UserWarning: Trying to unpickle estimator RandomForestRegressor from version 1.0.2 when using version 1.2.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/env/lib/python3.8/site-packages/sklearn/base.py:420: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(


Feature: 100/37


/env/lib/python3.8/site-packages/sklearn/base.py:299: UserWarning: Trying to unpickle estimator DecisionTreeRegressor from version 1.0.2 when using version 1.2.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/env/lib/python3.8/site-packages/sklearn/base.py:299: UserWarning: Trying to unpickle estimator RandomForestRegressor from version 1.0.2 when using version 1.2.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/env/lib/python3.8/site-packages/sklearn/base.py:420: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(


In [ ]:
end = time.time()
elapsed_time = end - start
minutes = int(elapsed_time // 60)
seconds = elapsed_time % 60
print(f"Elapsed time: {minutes} minutes and {seconds:.2f} seconds")

Elapsed time: 38 minutes and 38.30 seconds
